In [1]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.9 MB/

In [2]:
import cirq
import numpy as np

# Create two qubits
qubit1 = cirq.LineQubit(0)
qubit2 = cirq.LineQubit(1)

# Define a quantum circuit
circuit = cirq.Circuit(
    cirq.rx(np.pi / 4)(qubit1),  # Apply a rotation around X-axis by π/4 radians on qubit1
    cirq.CNOT(qubit1, qubit2),   # Apply a CNOT gate with qubit1 as control and qubit2 as target
    cirq.X(qubit2)               # Apply an X (NOT) gate to qubit2
)

# Print the circuit
print(circuit)

# Simulate the circuit
simulator = cirq.Simulator()
result = simulator.simulate(circuit)

# Display the result
print(result)


0: ───Rx(0.25π)───@───────
                  │
1: ───────────────X───X───
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: 0.924|01⟩ - 0.383j|10⟩

phase:
output vector: |⟩


In [21]:
import cirq
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess data (similar to the previous example)
data = load_iris()
X, y = data.data, data.target

# Only use two classes for binary classification and two features
X = X[y != 2][:, :2]
y = y[y != 2]

# Split and normalize the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the quantum circuit with Cirq
def create_quantum_circuit(data):
    # Create 2 qubits for the input data
    qubits = [cirq.GridQubit(0, i) for i in range(2)]

    # Define a simple quantum circuit
    circuit = cirq.Circuit()
    for i, value in enumerate(data):
        # Encode the input data into rotation angles on the qubits
        circuit.append(cirq.rx(value * np.pi).on(qubits[i]))
    
    # Add a CNOT gate for entanglement
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))

    # Measure the qubits
    circuit.append([cirq.measure(qubits[i], key=f'qubit_{i}') for i in range(2)])
    
    return circuit, qubits

# Simulate the quantum circuit and extract features
def run_quantum_circuit(inputs):
    # Assuming `inputs` has the shape (batch_size, 2)
    batch_size = inputs.shape[0]
    results = []
    
    for input_data in inputs:
        # Convert classical input data to qubits (using Cirq)
        circuit, qubits = create_quantum_circuit(input_data)
        
        # Run the circuit and get the results
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)
        
        # Extract measurements from the result
        measurement_0 = result.measurements['qubit_0'].flatten()[0]
        measurement_1 = result.measurements['qubit_1'].flatten()[0]
        
        # Combine the measurements into a single output
        combined_result = np.array([measurement_0, measurement_1])
        
        # Append to the batch results
        results.append(combined_result)
    
    # Convert to numpy array of the shape (batch_size, 2)
    return np.array(results, dtype=np.float32)

# Define a custom TensorFlow layer to integrate quantum processing
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()

    def call(self, inputs):
        # Expect input to be a batch of data
        output = tf.numpy_function(run_quantum_circuit, [inputs], tf.float32)
        output.set_shape((None, 2))  # Explicitly set the shape to (batch_size, 2)
        return output

# Build the hybrid model with TensorFlow
model = tf.keras.Sequential([
    QuantumLayer(),  # Quantum processing
    tf.keras.layers.Dense(32, activation='relu'),  # Classical layer

    tf.keras.layers.Dense(1, activation='sigmoid')  # Classical layer
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5890 - loss: 0.7048
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5093 - loss: 0.6877
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6470 - loss: 0.6684
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5215 - loss: 0.6873
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4983 - loss: 0.6912
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4362 - loss: 0.6761
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4383 - loss: 0.7161
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4908 - loss: 0.6862
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6552 - loss: 0.6718
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5119 - loss: 0.7074
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.5000 - loss: 0.6917
Test Accuracy: 50.00%
